# Training with HPO using Syne Tune Local Launcher Mode

## 1. Preparation

In [2]:
import sagemaker

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-pytorch-mnist'
role = sagemaker.get_execution_role()

In [3]:
!pip install -U torchvision

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [4]:
!pip install -U syne-tune

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [5]:
from torchvision.datasets import MNIST
from torchvision import transforms

local_dir = 'data'
MNIST.mirrors = ["https://sagemaker-sample-files.s3.amazonaws.com/datasets/image/MNIST/"]
MNIST(
    local_dir,
    download=True,
    transform=transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
    )
)

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )

In [6]:
inputs = sagemaker_session.upload_data(path='data', bucket=bucket, key_prefix=prefix)

In [16]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point='mnist.py',
    role=role,
    py_version='py3',
    framework_version='1.8.0',
    instance_count=1,
    instance_type='ml.c5.2xlarge',
    hyperparameters={'epochs': 1, 'backend': 'gloo'},
    source_dir='source_dir'
)

## 2. Training

In [17]:
from syne_tune import search_space
from syne_tune.optimizer.baselines import RandomSearch
from syne_tune.backend.sagemaker_backend.sagemaker_backend import SagemakerBackend
from syne_tune.stopping_criterion import StoppingCriterion
from syne_tune.tuner import Tuner

In [18]:
config_space = {
    'lr': search_space.loguniform(0.001, 0.1),
    'batch-size': search_space.choice([32, 64, 128, 256, 512])
}


In [19]:
scheduler = RandomSearch(
    config_space=config_space,
    mode='min',
    metric='test_loss',
    random_seed=31415927
)

In [20]:
backend = SagemakerBackend(
    sm_estimator=estimator,
    metrics_names=['test_loss'],
    inputs={'training': inputs}
)

In [21]:
stop_criterion = StoppingCriterion(max_wallclock_time=600)

In [25]:
tuner = Tuner(
    scheduler=scheduler,
    backend=backend,
    stop_criterion=stop_criterion,
    n_workers=3,
    sleep_time=5.0,
    tuner_name='hpo-hyperband'
)

In [ ]:
tuner.run()